<a href="https://colab.research.google.com/github/Theieyrre/Hate-Speech-NLP/blob/main/Fasttext_offensive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Offensive Labeling using fasttext

# Install fasttext

In [ ]:
# !git clone https://github.com/facebookresearch/fastText.git
# %cd fastText
# !pip install .

In [ ]:
!wget https://github.com/facebookresearch/fastText/archive/0.2.0.zip
!unzip 0.2.0.zip
%cd fastText-0.2.0
!make

--2020-08-06 07:35:58--  https://github.com/facebookresearch/fastText/archive/0.2.0.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/0.2.0 [following]
--2020-08-06 07:35:59--  https://codeload.github.com/facebookresearch/fastText/zip/0.2.0
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘0.2.0.zip’

0.2.0.zip               [     <=>            ]   4.10M  2.78MB/s    in 1.5s    

2020-08-06 07:36:01 (2.78 MB/s) - ‘0.2.0.zip’ saved [4304799]

Archive:  0.2.0.zip
7842495a4d64c7a3bb4339d45d6e64321d002ed8
   creating: fastText-0.2.0/
   creating: fastText-0.2.0/.circleci/
  inflating: fastText-0.2.0/.

## Imports

In [ ]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import Flatten
from keras.initializers import Constant

## Parameters

In [ ]:
# Data Parameters
h_na = '-'
h_train_filename = '/content/drive/My Drive/semeval/offenseval-tr-training-v1.tsv'
h_test_filename = '/content/drive/My Drive/semeval/offenseval-tr-testset-v1.tsv'
h_test_labels = '/content/drive/My Drive/semeval/offenseval-tr-labela-v1.tsv'
h_predict_filename = '/content/sentiment_OFF_100.csv'
h_t_sep = '\t'
h_c_sep = ','
h_index = 'id'
h_num_labels = 2
h_label = 'label'

# Fasttext parameters
h_max_word = 280
h_max_sequence = 1000
h_embedding_dim = 300

## Import and prepare data
Import train file

In [ ]:
df_train = pd.read_csv(h_train_filename, na_values=h_na, sep=h_t_sep)
df_train = df_train.rename(columns={"subtask_a": "label"})
df_train.head()

,id,tweet,label
0,20948,@USER en güzel uyuyan insan ödülü jeon jungkoo...,NOT
1,10134,"@USER Mekanı cennet olsun, saygılar sayın avuk...",NOT
2,23457,Kızlar aranızda kas yığını beylere düşenler ol...,NOT
3,18401,Biraz ders çalışayım. Tembellik ve uyku düşman...,NOT
4,17525,@USER Trezeguet yerine El Sharawy daha iyi olm...,NOT


Import test file and labels

In [ ]:
df_test = pd.read_csv(h_test_filename, na_values=h_na, sep=h_t_sep)
df_test.head()

,id,tweet
0,41993,@USER Sayın başkanım bu şekilde devam inşallah👏
1,23000,"Herkes gevşekliği kadar duyar kasıyor,hayat bö..."
2,42478,Olgun ilişkisi olan arkadaş size en güzel hedi...
3,21748,@USER @USER Burada atıp tutacağına o kötü koşu...
4,13607,@USER İşte o onur dediğin sende yok sorun o işte


In [ ]:
df_test_label = pd.read_csv(h_test_labels, na_values=h_na, sep=h_c_sep, names=['id', 'label'])
df_test_label.head()

,id,label
0,41993,NOT
1,23000,NOT
2,42478,NOT
3,21748,OFF
4,13607,OFF


### Add Labels to test dataframe

In [ ]:
df_test = df_test.merge(df_test_label, on='id')
df_test.head()

,id,tweet,label
0,41993,@USER Sayın başkanım bu şekilde devam inşallah👏,NOT
1,23000,"Herkes gevşekliği kadar duyar kasıyor,hayat bö...",NOT
2,42478,Olgun ilişkisi olan arkadaş size en güzel hedi...,NOT
3,21748,@USER @USER Burada atıp tutacağına o kötü koşu...,OFF
4,13607,@USER İşte o onur dediğin sende yok sorun o işte,OFF


### Train.txt


In [ ]:
f = open('train.txt', 'w')
for index, row in df_train.iterrows():
  f.write('__label__' + row.label + ' ' +  row.tweet.lower() +"\n")
f.close()

In [ ]:
!cat train.txt

Streaming output truncated to the last 5000 lines.
__label__NOT şu tablonun tek sebebi yönetimdir.. takım ruhsuz.. izlerken gözlerim kanıyo , keyif vermiyo.. yani her alınan top geri oynanırmı anlamıyorum.. fatih hocanın sacma sapan arayısları ayrı bi olay.. herseye ragmen yönetim bu senede sınıfta kaldı.. en uygun zamanda istifa etmelri grek
__label__OFF her sey daha kötu oldu aq :(((((
__label__NOT @user @user bizim düşman bir tane degil !
__label__NOT @user ya off güldüm 😄😄   zaytunga rakip olarak saytung olarak isminizi degisin tam olsun.
__label__NOT “bir gece başımızı alıp gitsek diyorum.     bir deniz kenarı mı olur, bir dağ başı mı olur.     kaçsak bu kalabalıktan.”
__label__OFF tüm para savur sonra kefen parasina göz koy ha nereye kadar aha onuda yedi̇n sonra ?
__label__OFF neymiş tudor geçen sene 15 haftada şu kadar puan toplamış da bilmem ne siz onu benim şeyime anlatın aq! tudorun elindeki kadro ile terimin elindeki kadro aynı mı amk adam 15 haftadır eren ve sinan denen fut

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Test.txt

In [ ]:
f = open('test.txt', 'w')
for index, row in df_test.iterrows():
  f.write('__label__' + row.label + ' ' +  row.tweet.lower() +"\n")
f.close()

NameError: ignored

In [ ]:
!cat test.txt

## Training

In [ ]:
!./fasttext supervised -input train.txt -output fasttext_model  -dim 2

Read 0M words
Number of words:  120958
Number of labels: 2
Progress: 100.0% words/sec/thread:  571422 lr:  0.000000 loss:  0.365936 ETA:   0h 0m


## Test

In [ ]:
!./fasttext test fasttext_model.bin test.txt

N	3515
P@1	0.834
R@1	0.834


# Train on full data

In [ ]:
f = open('train.txt', 'w')
for index, row in df_train.iterrows():
  f.write('__label__' + row.label + ' ' +  row.tweet.lower() +"\n")
f = open('test.txt', 'w')
for index, row in df_test.iterrows():
  f.write('__label__' + row.label + ' ' +  row.tweet.lower() +"\n")
f.close()

In [ ]:
!./fasttext supervised -input train.txt -output fasttext_model_all  -dim 2

Read 0M words
Number of words:  120958
Number of labels: 2
Progress: 100.0% words/sec/thread:  571526 lr:  0.000000 loss:  0.364472 ETA:   0h 0m


# Predict

In [ ]:
df_predict = pd.read_csv(h_predict_filename, na_values=h_na, sep=h_c_sep)

### Predict.txt

In [ ]:
f = open('predict.txt', 'w')
for index, row in df_predict.iterrows():
  f.write(row.tweet.lower() +"\n")
f.close()

In [ ]:
!./fasttext predict fasttext_model_all.bin predict.txt

__label__OFF
__label__OFF
__label__NOT
__label__NOT
__label__NOT
__label__NOT
__label__OFF
__label__OFF
__label__NOT
__label__NOT
__label__NOT
__label__OFF
__label__NOT
__label__NOT
__label__OFF
__label__OFF
__label__OFF
__label__NOT
__label__NOT
__label__NOT
__label__OFF
__label__OFF
__label__NOT
__label__OFF
__label__NOT
__label__NOT
__label__NOT
__label__NOT
__label__NOT
__label__NOT
__label__OFF
__label__OFF
__label__OFF
__label__OFF
__label__NOT
__label__NOT
__label__NOT
__label__OFF
__label__OFF
__label__OFF
__label__OFF
__label__NOT
__label__OFF
__label__NOT
__label__NOT
__label__OFF
__label__OFF
__label__NOT
__label__OFF
__label__OFF
__label__NOT
__label__NOT
__label__OFF
__label__NOT
__label__OFF
__label__NOT
__label__OFF
__label__OFF
__label__OFF
__label__OFF
__label__OFF
__label__OFF
__label__OFF
__label__OFF
__label__NOT
__label__NOT
__label__OFF
__label__NOT
__label__NOT
__label__NOT
__label__NOT
__label__NOT
__label__OFF
__label__NOT
__label__OFF
__label__NOT
__label__OFF

# Previous implementation code

## Tokenize data

In [ ]:
# tokenizer = Tokenizer(num_words=h_max_word)

### Tokenize Train data

In [ ]:
#tokenizer.fit_on_texts(df_train['tweet'])
#sequences = tokenizer.texts_to_sequences(df_train['tweet'])

#word_index_train = tokenizer.word_index
#print('Found %s unique tokens.' % len(word_index_train))
#data_train = pad_sequences(sequences, maxlen=h_max_sequence)

Found 100415 unique tokens.


In [ ]:
#labels_train = to_categorical(np.asarray(df_train['category']))

### Tokenize Test data

In [ ]:
#tokenizer.fit_on_texts(df_test['tweet'])
#sequences = tokenizer.texts_to_sequences(df_test['tweet'])

#word_index_test = tokenizer.word_index
#print('Found %s unique tokens.' % len(word_index_test))
#data_test = pad_sequences(sequences, maxlen=h_max_sequence)

Found 108088 unique tokens.


In [ ]:
#labels_test = to_categorical(np.asarray(df_test['category']))

# Load Model from fasttext

In [ ]:
#fasttext.util.download_model('tr', if_exists='ignore')
#ft = fasttext.load_model('cc.tr.300.bin')
#ft.get_dimension()

300

### Preparing embedding matrix.fast text
For train data

In [ ]:
#num_words = min(h_max_word, len(word_index_train) + 1)
#embedding_matrix_train = np.zeros((num_words, h_embedding_dim))
#for word, i in word_index_train.items():
#    if i >= h_max_word:
#        continue
#    if word in ft.vocab:
#      embedding_vector = ft[word]
#      embedding_vector = np.array(embedding_vector)
#      if embedding_vector is not None:
#          embedding_matrix_train[i] = embedding_vector

AttributeError: ignored

For test data

In [ ]:
#num_words = min(h_max_word, len(word_index_test) + 1)
#embedding_matrix_test = np.zeros((num_words, EMBEDDING_DIM))
#for word, i in word_index_test.items():
#    if i >= h_max_word:
#        continue
#    if word in ft.vocab:
#      embedding_vector = ft[word]
#      embedding_vector = np.array(embedding_vector)
#      if embedding_vector is not None:
#          embedding_matrix_train[i] = embedding_vector

# Define and train model

In [ ]:
#model = Sequential()
#model.add(Embedding(num_words,
#                            h_embedding_dim,
#                            embeddings_initializer=Constant(embedding_matrix_train),
#                            input_length=h_max_sequence,
#                            trainable=False))
#model.add(Flatten())
#model.add(Dense(512, activation='relu'))
#model.add(Dense(256, activation='relu'))
#model.add(Dense(128, activation='relu'))
#model.add(Dense(len(df_train['label']), activation='softmax'))

In [ ]:
#model.compile(loss='categorical_crossentropy',
#              optimizer='rmsprop',
#              metrics=['acc'])

In [ ]:
#print(model.summary())

In [ ]:
#model.fit(x_train, y_train,
#          batch_size=256,
#          epochs=1,
#          validation_data=(x_val, y_val))

# Performance on test data

In [ ]:
#loss, accuracy = model.evaluate(df_text['tweet'], df_test['label'], verbose=0)

#print('Accuracy: %f' % (accuracy))
#print('Loss: %f' % (loss))